# Algebra for Geometry Objects in PyMuPDF
To run this notebook, PyMuPDF version must be 1.19.6 or higher.

In [2]:
import fitz
assert tuple(map(int, fitz.VersionBind.split("."))) >= (1,19,6)

## Overview
PyMuPDF supports the 2-dimensional geometry objects as defined in MuPDF, resp. PDF specifications. Geometry objects are `Point`, `IRect`, `Rect`, `Quad` and `Matrix`. They all represent more or less what their names suggest them to be: they pertain to objects in plane geometry, mostly according to the requirements and terminology of PDF specification.
* A `Point` is represented by a pair of floats: `fitz.Point(x, y)`. In PyMuPDF they can be treated as 2-dimensional vectors, including component-wise **_addition_** and **_multiplication_** by numbers and matrices.
* `Rect` and `IRect` are specialized subsets of general rectangles: their edges always are parallel to the x- and the y-axis. While a `Rect` may be defined by float numbers, an `IRect` has integer coordinates. Further classifications include **_valid_**, **_empty_** or **_infinite_**, which imply certain behaviors and possibilities. Like points, rectangles can also be **_added_** to each other and can be **_multiplied_** by numbers and matrices. Because of their their special make-up, rectangles can be defined by just 2 points (or correspondingly 4 floats), that are diagonally opposite. By convention, (Py-) MuPDF uses the top-left and the bottom-right points (in **_this_** sequence) for this: `fitz.Rect(x0, y0, x1, y1)`. Using a different sequence, or the other two points leads to **_invalid_** rectangles.
* A `Quad` is a quadrilateral or tetragon. It is defined by its 4 corner points **_in the mandatory sequence_** upper-left (**ul**), upper-right (**ur**), lower-left (**ll**), lower-right (**lr**). Quads are much less frequently used than rectangles, but they do play an important role, e.g. for non-horizontal text.
* A `Matrix` represents a mapping (or "transformation") within objects of any of the above classes. Every geometry object above has method ``transform()``: `OBJ.transform(matrix)` maps OBJ (one of `Point`, `Rect`, `IRect`, `Quad`) to a new object of the same type, which then replaces OBJ. Matrices also support component-wise addition and mutiplication with each other via matrix method ``concat()``. A matrix is represented by 6 floats `fitz.Matrix(a, b, c, d, e, f)` arranged in a special structure of a row-major 3 x 3 matrix:
```
    / a b 0 \
    | c d 0 |
    \ e f 1 /
```
* The above objects are implemented as Python sequences, which makes it possible to access and modify their components also via index.
* The better part of functions and methods however **_only needs the underlying sequence_** of geometry objects. PyMuPDF therefore defines the notions "point-like", "rect-like", "quad-like" and "matrix-like" to indicate that a numerical sequence (tuple, list, array, ndarray ...) with the right length is sufficient.

## Algebraic Operations
PyMuPDF defines unary and binary operators that allow handling geometry objects comfortably and intuitively.

### Unary Operators
PyMuPDF defines a number of unary operators, i.e. involving only one geometry object. The result always is either a bool, a float, or a new object of the same type as the operand.
* `abs()` - *(float)* the Euclidean norm for `Point` and `Matrix`, and the area of `Rect` and `IRect` rectangles. For a `Quad` the product of its width and height.
* `norm()` - *(float)* the Euclidean norm.
* `bool()` - *(bool)* is `False` exactly if all components are zero.
* `hash()` - implemented as `hash(tuple(OBJ))`.
* `~m` - *(`Matrix`)* the inverse of matrix m. If not invertible, the matrix with all zeroes is returned.
* `+OBJ`, `-OBJ` - copy of the original. Negated components if `"-"` is used.

### Binary Operators
Binary operators involve two objects. The first operand determines the object type.

#### Addition, Subtraction
Use `+` and `-` to add / subtract objects of the **_same_** class. This works component-wise. The second operand must be **_like_** the first one - i.e. just a numerical sequence. The second operand may also be just a number, if all components should be modified with the same value. The result is always a **_new object._** The following example equalities apply analogously also to `Rect`, `IRect`, `Quad` and `Matrix` objects:

In [3]:
import fitz
(
 fitz.Point(1, 2) + fitz.Point(1, 1)
 == fitz.Point(1, 2) + (1, 1)
 == fitz.Point(1, 2) + 1
 == fitz.Point(2, 3)
)

True

#### Multiplication, Division
Use `*` and `/` to multiply points, rectangles, quads or matrices with a number or a matrix-like. For division, the number or matrix-like must obviously be invertible. The result is a **_new object._**

> Multiplication of matrices is not commutative - in the general case we have `m * n ≠ n * m`. Also, there exist non-zero matrices which are not invertible.

> There is a neutral `Matrix` under multiplication - `Identity` - such that for every matrix `m` we have `m * fitz.Identity == fitz.Identity * m == m`.

In [4]:
import fitz
fitz.Point(2, 4) / 2 == fitz.Point(1, 2)
(
    fitz.Point(1, 2) * fitz.Matrix(1, 2, 3, 4, 5, 6) == 
    fitz.Point(1, 2) * (1, 2, 3, 4, 5, 6)
)
(
    fitz.Point(1, 2) / fitz.Matrix(1, 2, 3, 4, 5, 6) ==
    fitz.Point(1, 2) / (1, 2, 3, 4, 5, 6)
)

True

In [5]:
import fitz
# this happens if dividing by a non-invertible matrix:
m = fitz.Matrix(1, 0, 1, 0, 1, 0)
try:
    fitz.Point(1, 2) / m
except ZeroDivisionError:
    print(f"{m} is not invertible")

Matrix(1.0, 0.0, 1.0, 0.0, 1.0, 0.0) is not invertible


#### Other Binary Operators
Checking whether geometry objects are equal is specialized to equality of their components.

Checking whether objects are contained in a rectangle or quad is fully supported.

Rectangles have special operators that allow building unions or intersections.

Relevant binary operators also support **_in-place notation._**

* Equality checks: geometry objects (and, resp., *-likes*) are regarded equal if `bool(a - b) == False`, where `b` may be a-like. This means that `fitz.Point(2, 4) == (2, 4)` returns `True`! But of course `(2, 4) != fitz.Point(2, 4)`!
* Containment checks: Classes `Rect`, `IRect` and `Quad` have method `OBJ.contains(x)`, which checks whether `x` is contained in `OBJ`, returning `True` or `False`. Object `x` may be any of **_point-like, rect-like_** or **_quad-like_**. The Python syntax `x in OBJ` is likewise supported.
* `&`, `|`: Rectangles (only) support **_unions_** and **_intersections_**. The expression `rect | x` ("union") delivers the **_smallest_** rectangle containing rectangle `rect` and rect-like / point-like `x`. The expression `rect & x` ("intersection") is the **_largest_** rectangle contained in both, rectangle `rect` and rect-like `x`. The following snippet creates the smallest rectangle that covers all points in a list:

In [6]:
import fitz
points = []  # a list of point-likes
for i in range(5):
    for j in range(3):
        points.append((i, j))
rect = fitz.EMPTY_RECT()  # start with the standard empty rectangle
for point in points:
    rect |= point  # in-place binary operators are supported!
print(rect)

Rect(0.0, 0.0, 4.0, 2.0)


As mentioned above, binary operators support **_in-place notation_** and allow elegant formulations like `rect *= (1, 2, 3, 4, 5, 6)`.

More examples:
* **_Middle point_** on the line connecting points p1 and p2: `mp = (p1 + p2) / 2`.
* Top-left **_quarter rectangle_** of a page: `rect = page.rect / 2`.
* Shift a **_rectangle to the right_** by 5 units: `rect += (5, 0, 5, 0)`.
* **_Enlarge a rectangle_** in every direction: `rect += (-5, -5, 5, 5)`.
* Compute page sub-rectangle, leaving a **_1-inch border_**: `rect = page.rect + (72, 72, -72, -72)`.

#### Setting Brackets
Objects of the same class are **_mathematical groups_** with respect to addition and thus allow arbitrary setting of brackets. I.e. the following is always true if a, b and c are all points, rectangles, quads or matrices:
```
a ± b ± c == (a ± b) ± c == a ± (b ± c)
```
The set of `Matrix` objects, with its additional multiplication, is a **_mathematical ring_**, which has the (left and right) **_distributive property:_**
```
(m1 ± m2) * m3 == m1 * m3 ± m2 * m3
m1 * (m2 ± m3) == m1 * m2 ± m1 * m3
```
Care must taken when setting **_brackets across different_** geometry classes! If r, r1, r2 are rectangles, λ, λ1, λ2 are numbers and m, m1, m2 are matrices, we have the following:

This is **_true:_**
```
(r1 ± r2) * λ == r1 * λ ± r2 * λ
```
**_But (*):_**
```
(r1 ± r2) * m ≠ r1 * m ± r2 * m
```
This is **_true:_**
```
r * m1 * m2 == (r * m1) * m2
r * λ1 * λ2 == (r * λ1) * λ2 == r * (λ1 * λ2)
```
**_But (*):_**
```
r * m1 * m2 ≠ r * (m1 * m2)
```
-----
(*) Inequalities refer to the general case.